In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install sentence-transformers bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bert_score
import warnings
warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized")

pd.set_option('display.max_rows', 1000)

In [ ]:
base_folder_path = '/content/drive/MyDrive/nlp/final'

os.listdir(base_folder_path)

['eval.ipynb',
 'QA_1001-1100.csv',
 'dataset-pdf',
 'GEMINI V.1',
 'eval',
 'prediction_with_clean_ds',
 'baseline',
 'attempts',
 'test_answer_list.csv',
 'answer_sampling_checkpoint_1_1001-1025.csv',
 'answer_sampling_checkpoint_2_1026-1050.csv',
 'answer_sampling_checkpoint_3_1051-1075.csv',
 'answer_sampling_question.csv',
 'answer_sampling_checkpoint_4_1076-1100.csv',
 'ans_gen.ipynb',
 'QA_sampling_1001-1100.csv',
 'similarity_scores_with_cos_sim.csv',
 'mean_similarity_scores_by_paper.csv',
 'mean_similarity_scores_by_model.csv']

In [ ]:
df = pd.read_csv(os.path.join(base_folder_path, 'QA_sampling_1001-1100.csv'))

In [ ]:
answer_columns = ['answer_from_src', 'answer_from_sum', 'answer_from_lora', 'answer_from_baseline']

model = SentenceTransformer('all-MiniLM-L6-v2')

similarities_cos_sim = []

for index, row in tqdm(df.iterrows(), total=len(df)):
  actual = str(row['actual_answer']) if pd.notnull(row['actual_answer']) else ""
  actual_embedding = model.encode(actual, convert_to_tensor=True)

  result = {
    'q_index': row['q_index'],
    'paper_id': row['paper_id'],
    'section': row['section'],
    'question': row['question']
  }

  for col in answer_columns:
    other_answer = str(row[col]) if pd.notnull(row[col]) else ""
    other_embedding = model.encode(other_answer, convert_to_tensor=True)
    score = util.cos_sim(actual_embedding, other_embedding).item()
    result[f'similarity_to_{col}'] = (score + 1)/2

  similarities_cos_sim.append(result)

similarity_cos_sim_df = pd.DataFrame(similarities_cos_sim)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

100%|██████████| 3755/3755 [02:11<00:00, 28.64it/s]


In [ ]:
similarity_cos_sim_df

,q_index,paper_id,section,question,similarity_to_answer_from_src,similarity_to_answer_from_sum,similarity_to_answer_from_lora,similarity_to_answer_from_baseline
0,0,1001,Abstract:,What type of data does the algorithm take adva...,0.641470,0.641470,0.587084,0.802398
1,1,1001,Section 1.1: De-duplication & Encryption,What does CAS stand for?,0.516203,1.000000,0.573740,1.000000
2,2,1001,Section 1.1: De-duplication & Encryption,What is there a good deal of between the data ...,0.646033,0.796798,0.792813,1.000000
3,3,1001,Section 1.2: A Solution,What is the name of the software that overcome...,0.549079,1.000000,0.458906,0.927847
4,4,1001,Section 1.2: A Solution,What is the algorithm capable of identifying i...,0.855413,0.954464,0.573829,1.000000
...,...,...,...,...,...,...,...,...
3750,3750,1100,Section 6: Related Work,What is the coarse-grained version of informat...,0.598562,0.548011,0.547108,0.558744
3751,3751,1100,Section 7: Conclusion,What language was the enterprise-grade applica...,1.000000,1.000000,0.641785,1.000000
3752,3752,1100,Section 7: Conclusion,What does DBTaint do when database clients ret...,0.556618,0.866724,0.548637,0.599315
3753,3753,1100,Section 7: Conclusion,What does DBTaint require to the database?,0.589220,0.872138,0.765295,0.872138


In [ ]:
similarity_cos_sim_df.to_csv(os.path.join(base_folder_path, 'similarity_scores_with_cos_sim.csv'), index=False)

In [ ]:
similarity_cos_sim_columns = [
    'similarity_to_answer_from_src',
    'similarity_to_answer_from_sum',
    'similarity_to_answer_from_lora',
    'similarity_to_answer_from_baseline'
]

In [ ]:
mean_by_paper_cos_sim = similarity_cos_sim_df.groupby('paper_id')[similarity_cos_sim_columns].mean().reset_index()

display(mean_by_paper_cos_sim)

,paper_id,similarity_to_answer_from_src,similarity_to_answer_from_sum,similarity_to_answer_from_lora,similarity_to_answer_from_baseline
0,1001,0.650723,0.792363,0.615739,0.883459
1,1002,0.737537,0.798909,0.712113,0.776828
2,1003,0.689459,0.827203,0.643573,0.728403
3,1004,0.690570,0.731233,0.660676,0.749896
4,1005,0.649565,0.838508,0.668440,0.750003
5,1006,0.715633,0.797441,0.648680,0.720530
6,1007,0.723719,0.775616,0.706089,0.783056
7,1008,0.685136,0.794650,0.616430,0.933350
8,1009,0.776571,0.804528,0.687171,0.762956
9,1010,0.667541,0.780062,0.664824,0.753263


In [ ]:
mean_by_paper_cos_sim.to_csv(os.path.join(base_folder_path, 'mean_similarity_cos_sim_scores_by_paper.csv'), index=False)

In [ ]:
mean_each_model_cos_sim = mean_by_paper_cos_sim[similarity_columns].mean().reset_index()
mean_each_model_cos_sim.columns = ['similarity_type', 'score']

mean_each_model_cos_sim

,similarity_type,score
0,similarity_to_answer_from_src,0.688609
1,similarity_to_answer_from_sum,0.785756
2,similarity_to_answer_from_lora,0.657397
3,similarity_to_answer_from_baseline,0.783169


In [ ]:
mean_each_model_cos_sim.to_csv(os.path.join(base_folder_path, 'mean_similarity_cos_sim_scores_by_model.csv'), index=False)

In [ ]:
answer_columns = ['answer_from_src', 'answer_from_sum', 'answer_from_lora', 'answer_from_baseline']

similarities_bert = []

for index, row in tqdm(df.iterrows(), total=len(df)):
    actual = str(row['actual_answer']) if pd.notnull(row['actual_answer']) else ""

    result = {
        'q_index': row['q_index'],
        'paper_id': row['paper_id'],
        'section': row['section'],
        'question': row['question']
    }

    for col in answer_columns:
        other_answer = str(row[col]) if pd.notnull(row[col]) else ""

        P, R, F1 = bert_score.score([actual], [other_answer], lang="en", model_type="bert-base-uncased")

        result[f'bert_score_to_{col}_F1'] = F1.item()

    similarities_bert.append(result)

similarity_bert_df = pd.DataFrame(similarities_bert)

  0%|          | 0/3755 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

100%|██████████| 3755/3755 [3:19:25<00:00,  3.19s/it]


In [ ]:
similarity_bert_df.to_csv(os.path.join(base_folder_path, 'similarity_scores_with_bert.csv'), index=False)

In [ ]:
similarity_bert_df

,q_index,paper_id,section,question,bert_score_to_answer_from_src_F1,bert_score_to_answer_from_sum_F1,bert_score_to_answer_from_lora_F1,bert_score_to_answer_from_baseline_F1
0,0,1001,Abstract:,What type of data does the algorithm take adva...,0.512832,0.512832,0.351639,0.595297
1,1,1001,Section 1.1: De-duplication & Encryption,What does CAS stand for?,0.287638,1.000000,0.273780,1.000000
2,2,1001,Section 1.1: De-duplication & Encryption,What is there a good deal of between the data ...,0.430445,0.672815,0.548870,1.000000
3,3,1001,Section 1.2: A Solution,What is the name of the software that overcome...,0.377373,1.000000,0.572994,0.656204
4,4,1001,Section 1.2: A Solution,What is the algorithm capable of identifying i...,0.655613,0.777037,0.408688,1.000000
...,...,...,...,...,...,...,...,...
3750,3750,1100,Section 6: Related Work,What is the coarse-grained version of informat...,0.396484,0.344697,0.454226,0.392251
3751,3751,1100,Section 7: Conclusion,What language was the enterprise-grade applica...,1.000000,1.000000,0.473963,1.000000
3752,3752,1100,Section 7: Conclusion,What does DBTaint do when database clients ret...,0.456066,0.536575,0.505508,0.441705
3753,3753,1100,Section 7: Conclusion,What does DBTaint require to the database?,0.395116,0.903950,0.631935,0.903950


In [ ]:
similarity_bert_columns = [
    'bert_score_to_answer_from_src_F1',
    'bert_score_to_answer_from_sum_F1',
    'bert_score_to_answer_from_lora_F1',
    'bert_score_to_answer_from_baseline_F1'
]

In [ ]:
mean_by_paper_bert = similarity_bert_df.groupby('paper_id')[similarity_bert_columns].mean().reset_index()

display(mean_by_paper_bert)

,paper_id,bert_score_to_answer_from_src_F1,bert_score_to_answer_from_sum_F1,bert_score_to_answer_from_lora_F1,bert_score_to_answer_from_baseline_F1
0,1001,0.488042,0.678693,0.441380,0.790300
1,1002,0.622165,0.672049,0.553131,0.673846
2,1003,0.551437,0.722699,0.423825,0.595082
3,1004,0.553260,0.564712,0.516756,0.623676
4,1005,0.499019,0.699883,0.536388,0.636133
5,1006,0.567610,0.675457,0.475306,0.577512
6,1007,0.585244,0.660923,0.564735,0.649349
7,1008,0.560703,0.682884,0.398166,0.868055
8,1009,0.625277,0.678886,0.535985,0.619270
9,1010,0.584643,0.688757,0.566647,0.648384


In [ ]:
mean_by_paper_bert.to_csv(os.path.join(base_folder_path, 'mean_similarity_bert_scores_by_paper.csv'), index=False)

In [ ]:
mean_each_model_bert = mean_by_paper_bert[similarity_bert_columns].mean().reset_index()
mean_each_model_bert.columns = ['similarity_type', 'F1 score']

mean_each_model_bert

,similarity_type,F1 score
0,bert_score_to_answer_from_src_F1,0.548684
1,bert_score_to_answer_from_sum_F1,0.666849
2,bert_score_to_answer_from_lora_F1,0.501394
3,bert_score_to_answer_from_baseline_F1,0.670671


In [ ]:
mean_each_model_bert.to_csv(os.path.join(base_folder_path, 'mean_similarity_bert_scores_by_model.csv'), index=False)